In [20]:
from GetData.GetDataMultiDimensions import get_test_data, get_train_data
from GetData.GetDataOneDimension import getData

In [73]:
for dataset in ["BasicMotions",
    "ERing",
    "EyesOpenShut",
    "FingerMovements",
    "Epilepsy",
    "JapaneseVowels",
    "Libras",
    "NATOPS",
    "RacketSports"]:
    X_train_BasicMotions, y_train_BasicMotions, X_test_BasicMotions, y_test_BasicMotions = getData(dataset, '../Data/OneDimension/')
    print(X_train_BasicMotions[1].shape)

(6, 100)
(4, 65)
(14, 128)
(28, 50)
(3, 206)
(12, 26)
(2, 45)
(24, 51)
(6, 30)


In [74]:
import numpy as np
def subsequences(time_series, k):
    time_series = np.asarray(time_series)
    n = len(time_series)
    shape = (n - k + 1, k) + time_series.shape[1:]
    if time_series.ndim == 1:
        strides = time_series.strides * 2
    elif time_series.ndim == 2:
        strides = time_series.strides[0], time_series.strides[0], time_series.strides[1]
    return np.lib.stride_tricks.as_strided(time_series, shape=shape, strides=strides)

In [75]:
import numpy as np
a = np.array([1,2,3,4,5,6,7,8,9,10])
subsequences(a, 2)

array([[ 1,  2],
       [ 2,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10]])

In [76]:
b = np.array([[1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [7,8], [8,9], [9,10]])
b.shape

(9, 2)

In [80]:
subs_xs = subsequences(b, 4)
subs_xs

array([[[ 1,  2],
        [ 2,  3],
        [ 3,  4],
        [ 4,  5]],

       [[ 2,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6]],

       [[ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7]],

       [[ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8]],

       [[ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9]],

       [[ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10]]])

In [40]:
import numpy as np

def split_into_subsequences_3d(data, shape):
    strides = data.strides
    shape = tuple(shape)
    strides = tuple(stride * shape[-1] for stride in strides) + strides
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)


In [41]:

# Sử dụng hàm
data = np.arange(24).reshape((2, 3, 4))  # Một mảng 3 chiều (2x3x4)
subsequences = split_into_subsequences_3d(data, (2, 1, 2))  # Chia thành các chuỗi con (2x1x2)
print(data,subsequences)

ValueError: mismatch in length of strides and shape

In [15]:
subs_xs = subsequences(X_train_BasicMotions[1], 2)
subs_xs

ValueError: mismatch in length of strides and shape

In [ ]:
def masking_map_linear_sub_sequence(
    xs,
    xt,
    ratio=0.1,
    sub_ratio=0.1,
    eps=1e-10,
    reg=0.0001,
    max_iterations=100000,
    thres=1e-5,
    algorithm="linear_programming",
):
    sub_length = int(np.floor(min(len(xs), len(xt)) * sub_ratio))
    subs_xs = subsequences(xs, sub_length)
    subs_xt = subsequences(xt, sub_length)
    p = np.ones(len(subs_xs)) / len(subs_xs)
    q = np.ones(len(subs_xt)) / len(subs_xt)
    C = cost_matrix_aw(subs_xs, subs_xt)
    C /= C.max() + eps
    M = create_mask(C, ratio)
    if algorithm == "linear_programming":
        pi = lp(p, q, C, M)
    elif algorithm == "sinkhorn":
        pi = sinkhorn_log_domain(p, q, C, M, reg, max_iterations, thres)
    else:
        raise ValueError("algorithm must be 'linear_programming' or 'sinkhorn'!")
    cost = np.sum(pi * C)
    return cost

In [ ]:
from MaskingMap.MaskingMapLinear import masking_map_linear
from MaskingMap.knn import knn_masking_map_linear, knn_masking_map_auto_weighted_short, knn_masking_map_auto_weighted

In [ ]:
accuracy, time = knn_masking_map_linear(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, lamb=2)
print(accuracy, time)

100%|██████████| 30/30 [13:45<00:00, 27.53s/it]


0.82 826.0064129829407


In [ ]:
# with open('../Config/OneDimensionalDataset.txt', 'r') as file:
#     # [print(line.strip()) for line in file]
#     for line in file:
#         print(line.strip())

import json
with open('../Config/OneDimension.json', 'r') as file:
    data = json.load(file)
for data in data["masking_map_linear"]["parameters"]:
    print(data["k"])


1


In [ ]:
accuracy = knn_masking_map_auto_weighted(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.2, lamb=3)
print(accuracy)

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [38:50<00:00, 77.68s/it]

0.7266666666666667


In [ ]:
accuracy = knn_masking_map_auto_weighted(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.1, lamb=5)
print(accuracy)

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [36:43<00:00, 73.44s/it]

0.7333333333333333


In [ ]:
accuracy = knn_masking_map_auto_weighted(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.2, lamb=3, algorithm="sinkhorn")
print(accuracy)

100%|██████████| 30/30 [4:59:53<00:00, 599.78s/it]  

0.74


In [ ]:
accuracy = knn_masking_map_auto_weighted(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.1, lamb=5, algorithm="sinkhorn")
print(accuracy)

100%|██████████| 30/30 [3:25:51<00:00, 411.70s/it]  

0.7133333333333334


In [ ]:
accuracy = knn_masking_map_auto_weighted_short(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.1, lamb=10)
print(accuracy)

c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.7933333333333333


In [ ]:
accuracy = knn_masking_map_auto_weighted_short(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.1, lamb=20)
print(accuracy)

c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.7733333333333333


In [ ]:
accuracy = knn_masking_map_auto_weighted_short(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.2, lamb=20)
print(accuracy)

c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.76


In [ ]:
accuracy = knn_masking_map_auto_weighted_short(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,ratio=0.3, lamb=10)
print(accuracy)

c:\Users\hoang\OneDrive\Documents\lab\AutoWeighted\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.7333333333333333
